In [1]:
# Step 1 : Creating Data 
import cv2 
import numpy as np

# Loading Haar_face_cascade_classifier 
face_classfier=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Loading functions 

def face_extractor(img):
    # This function will detect the faces and return the cropped faces
    # If no face is detected,it returns the input image.
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_classfier.detectMultiScale(gray,1.3,5)
    
    if faces is ():
        return None
    
    # Cropping all the faces found
    
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h,x:x+w]
    return cropped_face

# Initializing webcam
cap=cv2.VideoCapture(0)
count=0

# Collect 100 samples of your face from webcam input
while True:
    
    
    ret,frame=cap.read()
    if face_extractor(frame) is not None:
        count+=1
        face=cv2.resize(face_extractor(frame),(200,200))
        face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        
        # Save file in specified directory with unique name 
        file_name_path='./dataset_faces_2/'+ str(count) + '.jpg'
        cv2.imwrite(file_name_path,face)
        
        # Putting count on images and display live count
        cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
        cv2.imshow('Face Cropped',face)
    else:
        print("Face not found")
        pass
    
    if cv2.waitKey(1)==13 or count==100 :
        # 13 is the Enter key
        break
cap.release()
cv2.destroyAllWindows()
print("Collecting samples completed !")

<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-1-fd79bbd57553>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [2]:
# Step 2 - Training the model
cap.release()

In [3]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Getting the training data we previously made
data_path='./dataset_faces_2/'
onlyfiles=[f for f in listdir(data_path) if isfile(join(data_path,f))]

# Creating arrays for training data and labels 
Training_Data, Labels = [], []

# Open training images in our datapath
# Creating a numpy array for training data 

for i, files in enumerate(onlyfiles):
    image_path=data_path + onlyfiles[i]
    images=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images,dtype=np.uint8))
    Labels.append(i)
# Creating a numpy array for both training data and labels
Labels=np.asarray(Labels,dtype=np.int32)
# Initialize facial recognizer
# model=cv2.face.createLBPHFaceRecognizer()
# NOTE : For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model=cv2.createLBPHFacerecognizer()

astha_model=cv2.face.LBPHFaceRecognizer_create()

# Training the model
astha_model.train(np.asarray(Training_Data),np.asarray(Labels))
print("Model Trained successfully")




Model Trained successfully


In [4]:
# Step 3 - Running our Facial Recognition

In [5]:
import cv2
import numpy as np
import os

face_classifier=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_detector(img,size=0.5):
    # Converting image to grayscale
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    if faces is ():
        return img, []
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi,(200,200))
    return img,roi
# Open Webcam
cap=cv2.VideoCapture(0)
while True:
    ret, frame=cap.read()
    image, face=face_detector(frame)
    
    try:
        face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        
        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        
        results=astha_model.predict(face)
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 90:
            os.system("aws run ec2-instances --image-id  ami-06a0b4e3b7eb7a300 --instance-type t2.micro --subnet-id subnet --count 1 --security-group-ids sg-06af44cdc039a8b87 --subnet-id subnet-af6168c7 --key-name key > ec2.txt")             
            print("Instance launched....!")
            os.system("aws ec2 create-volume --availability-zone ap-south-1a --size 5 --volume-type gp2 --tag-specification ResourceType=volume,Tags=[{Key=face,value=volume}] > ebs.txt ")
            print("Volume created !")
            
            
            time.sleep(120)
            ec2_id=open("av.txt",'r').read().split(',')[3].split(':')[1].split('"')[1]
            ebs_id=open("ebs.txt",'r').read().split(',')[6].split(':')[1].split('"')[1]
            os.system("aws ec2 attach-volume --instance-id " + ec2_id + "-- volume-id" + ebs_id +
                     "device /dev/xvdf")
            print("Volume attached to the instance")
            break
                      
                      
        else: 
            cv2.putText(image, "I dont know...who r u", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()     


            
    
    
    
    

<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-5-2289721c3788>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Instance launched....!
Volume created !
Instance launched....!
Volume created !
Instance launched....!
Volume created !
Instance launched....!
Volume created !
Instance launched....!
Volume created !
Instance launched....!
Volume created !
Instance launched....!
Volume created !
